# Segmenting and Clustering Neighborhoods in Toronto

## Question 1 - Scraping Wikipedia

In [1]:
import pandas as pd
print('Pandas library imported.')
import requests
print('Requests library imported.')
from bs4 import BeautifulSoup
print('BeautifulSoup library imported.')

print('All libraries imported.')

Pandas library imported.
Requests library imported.
BeautifulSoup library imported.
All libraries imported.


## Scrape the Wikipedia page and transform it into a dataframe with BeautifulSoup and Pandas

In [2]:
result = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(result.content,'lxml')
table = soup.find_all('table')[0] 
neighborhoods = pd.read_html(str(table))[0]

## Cleaning the dataframe

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.

In [3]:
# Rename columns according to the assignment
neighborhoods.columns = ['PostalCode', 'Borough', 'Neighborhood']

neighborhoods.head()

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
# Remove lines for which column Borough has value "Not assigned"
neighborhoods = neighborhoods[neighborhoods['Borough']!='Not assigned']

neighborhoods.head()

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [5]:
# The Wikipedia data is already grouped by postal code (update since the creation of the assignement?), we just have to replace " /" by ","
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].str.replace(' /', ',')

neighborhoods.head()

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [6]:
neighborhoods.loc[neighborhoods['Neighborhood'] =='Not assigned' , 'Neighborhood'] = neighborhoods['Borough']

neighborhoods.head()

Final cleaning.

In [7]:
# Reset index
neighborhoods = neighborhoods.reset_index()
neighborhoods.drop(['index'], axis='columns', inplace=True)

neighborhoods.head()

## Display result

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
# Print the shape
print('The dataframe has the following shape: {}.'.format(
        neighborhoods.shape,
    )
)

The dataframe has the following shape: (103, 3).


## Question 2 - Get latitude and logitude of each postal code

## Install and import libraries

In [10]:
#!conda install -c conda-forge geocoder
import geocoder
print('Geocoder library imported.')

print('All libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    python_abi:    

## Get latitude and longitude

### Trying with Google Geocoding

Create a function that will loop until coordinates are obtained from Geocoder.

In [ ]:
def get_coordinates_from_geocoder(address):
    # initialize your variable to None
    coordinates = None

    # loop until you get the coordinates
    index = 1
    while(coordinates is None):
        print('Iteration {}'.format(index))
        g = geocoder.google(address)
        coordinates = g.latlng
        index = index + 1
        
        if index == 25:
            break

        return coordinates

Add 2 new columns to the dataframe.

In [ ]:
neighborhoods['Latitude'] = 0
neighborhoods['Longitude'] = 0

Use the function created above to create a new data frame with coordinates.

In [ ]:
for index in range(len(neighborhoods)):
    address = '{}, Toronto, Ontario'.format(neighborhoods.loc[index, 'PostalCode'])
    print('-----')
    print('row {} beginning: {}'.format(index, address))
    coordinates = get_coordinates_from_geocoder(address)
    if coordinates is not None:
        neighborhoods.loc[index, 'Latitude'] = coordinates[0]
        neighborhoods.loc[index, 'Longitude'] = coordinates[1]
    else:
        print("Error with Google Geocoding")
    print('{}%: row {} completed'.format(round(index/len(neighborhoods.index)*100, 1), index))

### Because Google Geocoding is not working, we are using the provided csv

Download the csv file.

In [ ]:
!wget -q -O 'Toronto_coordinates.csv' http://cocl.us/Geospatial_data
toronto_coordinates = pd.read_csv('Toronto_coordinates.csv')
toronto_coordinates.head()

## Create the new dataframe with coordinates

Merge the 2 dataframes.

In [ ]:
# Prepare the coordinates dataframe to be merged by changing the column names so they correspond to the neighborhood dataframe
toronto_coordinates.columns=['PostalCode','Latitude','Longitude']

# Remove the 2 columns created during the attempt to make Google Geocoding work to avoid duplicates
neighborhoods.drop(['Latitude'], axis='columns', inplace=True)
neighborhoods.drop(['Longitude'], axis='columns', inplace=True)

# Merge
neighborhoods = pd.merge(neighborhoods, toronto_coordinates[['PostalCode','Latitude', 'Longitude']], on='PostalCode')

## Display the dataframe

In [ ]:
neighborhoods

# Question 3 - Cluster the neighborhoods in Toronto 

## Install and import libraries

In [ ]:
import numpy as np # library to handle data in a vectorized manner
print('Numpy library imported.')

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Geopy library imported.')

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Matplotlib libraries imported.')

# import k-means from clustering stage
from sklearn.cluster import KMeans
print('KMeans library imported.')

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Folium library imported.')

print('Libraries imported.')

Numpy library imported.
Geopy library imported.
Matplotlib libraries imported.
KMeans library imported.
Solving environment: | 

## Create a map of Toronto with the neighborhood

Use geopy library to get the latitude and longitude values of Toronto.

In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

Create a map of Toronto with neighborhoods superimposed on top.

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

Display the map.

In [ ]:
map_toronto

## Obtain venues from Foursquare for each neighborhood

Define Foursquare Credentials and Version.

In [11]:
# The code was removed by Watson Studio for sharing.

Get the top 100 venues for each neighborhood within a radius of 500 meters.

In [12]:
# Initialize some variables
LIMIT = 100
RADIUS = 500

In [14]:
# First we create a function to get the venues for a specific neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=RADIUS):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe.

In [ ]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

print(toronto_venues.shape)
toronto_venues.head()

Let's check how many venues were returned for each neighborhood.

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

## Analyze Each Neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
toronto_grouped.shape

Print each neighborhood along with the top 5 most common venues.

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Convert that into a *pandas* dataframe.

In [ ]:
# Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Cluster neighborhood

Initialize some variables.

In [ ]:
# Set number of clusters
KCLUSTERS = 5

Run *k*-means to cluster the neighborhood.

In [ ]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=KCLUSTERS, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

Finally, let's visualize the resulting clusters.

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# The END